In [98]:
import pandas as pd
import numpy as np

read csv from github

In [99]:
url = "https://raw.githubusercontent.com/philippdrebes/sda-hockey-c2d/main/dummy%20data/dummy_data_complete.csv"
data = pd.read_csv(url, sep=",")
data = data.drop("Unnamed: 0", axis=1)

group dataframe by player id

In [100]:
avg_player = data.groupby(['Player ID', 'Position'], as_index=False).mean()
avg_player = avg_player.drop("Group Id", axis=1)


group dataframe by position

In [101]:
avg_position = data.groupby('Position', as_index=False).mean()
avg_position = avg_position.drop("Group Id", axis=1).drop("Player ID", axis=1)

create df with aggregated values (player values relative to average values of position)

In [102]:
rows = avg_player.shape[0]
values = []

for i in range(rows):
    dict_values = {}
    pos = avg_player.at[i,"Position"]
    dict_values["Player ID"] = avg_player.at[i,"Player ID"]
    for col in avg_player.columns:
        if col != "Player ID" and col != "Position":
            relative = avg_player.at[i, col] / avg_position.at[pos-1, col]
            dict_values[col] = relative
    values.append(dict_values)

agg_df = pd.DataFrame(values)      


    

evaluate agg_df and create two dfs (strengths and weaknesses) filtering only top and bottom 3 values per player

In [103]:
cols_for_evaluation = [col for col in agg_df.columns if col != "Player ID"]

# Perform the nlargest calculation on the other columns
strengths_df = (agg_df[cols_for_evaluation]
                  .stack()
                  .groupby(level=0)
                  .nlargest(3)
                  .unstack()
                  .reset_index(level=1, drop=True)
                  .reindex(columns=cols_for_evaluation))

# Reinsert the "Player ID" column into strengths_df
strengths_df.insert(0, 'Player ID', agg_df['Player ID'])

# Perform the nlargest calculation on the other columns
weaknesses_df = (agg_df[cols_for_evaluation]
                  .stack()
                  .groupby(level=0)
                  .nsmallest(3)
                  .unstack()
                  .reset_index(level=1, drop=True)
                  .reindex(columns=cols_for_evaluation))

# Reinsert the "Player ID" column into strengths_df
weaknesses_df.insert(0, 'Player ID', agg_df['Player ID'])

create dictionaries containing the weaknesses and strenghts for each player with their actual value

In [104]:
rows = strengths_df.shape[0]

strengths_dict = {}
weaknesses_dict = {}

for i in range(rows):
    strengths_dict[strengths_df.at[i, "Player ID"]] = {}
    for col in strengths_df.columns:
        if col != "Player ID" and not np.isnan(strengths_df.loc[i, col]):
            strengths_dict[strengths_df.at[i, "Player ID"]][col] = avg_player.at[i, col]

for i in range(rows):
    weaknesses_dict[weaknesses_df.at[i, "Player ID"]] = {}
    for col in weaknesses_df.columns:
        if col != "Player ID" and not np.isnan(weaknesses_df.loc[i, col]):
            weaknesses_dict[weaknesses_df.at[i, "Player ID"]][col] = avg_player.at[i, col]
            

In [105]:
print(strengths_dict)
print(weaknesses_dict)

{63: {'Distance / min (m)': 154.89731178720254, 'Speed (Ø) (km/h)': 12.515624362002374, 'Decelerations': 9.5}, 64: {'High Metabolic Power Distance (m)': 735.75, 'High Metabolic Power Distance / min (m)': 26.222071207772213, 'Deceleration (max.) (m/s²)': -3.800282490008583}, 65: {'Distance (m)': 5721.25, 'Accelerations (high)': 4.5, 'Acceleration Load (max.)': 9.089854669544192}, 66: {'Distance (m)': 5474.75, 'High Metabolic Power Distance (m)': 740.75, 'Acceleration Load (max.)': 8.98665359906896}, 67: {'Time on Ice (s)': 2529.25, 'Speed (max.) (km/h)': 22.926330264149545, 'Deceleration (max.) (m/s²)': -3.812561704282038}, 68: {'Distance (m)': 5465.75, 'Distance / min (m)': 152.681731449089, 'Time on Ice (s)': 2385.25}, 69: {'High Metabolic Power Distance / min (m)': 26.121472497493826, 'Accelerations (high)': 4.75, 'Deceleration (max.) (m/s²)': -3.9094588423428247}, 70: {'Time on Ice (s)': 2476.25, 'Accelerations (high)': 5.0, 'Acceleration (max.) (m/s²)': 2.5173715748045677}, 71: {'D